0. **Improve your business vision**

In [ ]:
vision = """

Write your vision here...

"""

additional_prompt = ""

In [ ]:
import pandas as pd
from importlib import reload

import lib.refine_vision as this
reload(this)

res = this.chain.invoke({"vision": vision, "additional_prompt":additional_prompt})
vision = {"vision": res.content.replace("\n", "")}
pd.DataFrame([vision]).to_csv('./output/vision.csv',index=False)

In [ ]:
print(res.content)

1. **Discovery of Main Jobs and Job Performers**

In [ ]:
additional_prompt = ""

In [ ]:
import json
import pandas as pd
from importlib import reload

import lib.discover_performers_and_jobs as this
reload(this)

res = this.chain.invoke({"vision": vision, "additional_prompt": additional_prompt})
res_dict = json.loads(res.additional_kwargs['function_call']['arguments'])['performers_and_jobs_list']
performers_and_jobs = pd.DataFrame(res_dict)

performers_and_jobs.to_csv('./output/performers_and_jobs.csv',index=False)

performers_and_jobs

2. **Synthesis of Job Performers**

In [ ]:
selected_job_performers = ""
selected_main_job = ""
number = ""
additional_prompt   = ""

In [ ]:
import json
import pandas as pd
from importlib import reload

import lib.generate_performers as this
reload(this)

res = this.chain.invoke({"number": number, "job_performers": selected_job_performers, "additional_prompt": additional_prompt})

performers = pd.DataFrame(json.loads(res.content)["job_performers"])
performers.to_csv('./output/performers.csv',index=False)

performers

3. **Automated Interviews**

In [ ]:
import json

from importlib import reload

import lib.interview_performers as this
reload(this)

interviews = []
for index, row in performers.iterrows():
    performer = row.to_json()

    # main_job_related = f"""Answer the following question focusing and thinking about the main job {selected_main_job} applied to what you are doing as {row['Profession']}.
    # If the main job {selected_main_job} is not related to you, simply answer \"This is not my main job\" and skip the question."""

    main_job_related = ""

    res = this.chain.invoke({"job_performer": performer, "main_job_related": main_job_related})
    res_dict = json.loads(res.additional_kwargs['function_call']['arguments'])
    
    interview = pd.DataFrame(res_dict["interview"])
    
    job_performer_dict = row.to_dict()

    job_performer_dict.update({"interview": interview})

    interviews.append(job_performer_dict)

In [ ]:
import os
outdir = './output/interviews'
if not os.path.exists(outdir):
    os.mkdir(outdir)

for interview in interviews:
    interview["interview"].to_csv(
        os.path.join(outdir,"_".join((
        interview["Name"].replace(" ", ""),
        interview["Profession"].replace(" ", ""),
        interview["Industry"].replace(" ", ""),
        interview["Experience_level"].replace(" ", ""),
        ))+".csv"),
        index=False)

4. **Analyze Interviews**

In [ ]:
import json
import pandas as pd
from importlib import reload

import lib.analyse_interviews as this
reload(this)

analysis_list = []
for interview in interviews:
    res = this.chain.invoke({"interview":interview})
    res_dict = json.loads(res.additional_kwargs['function_call']['arguments'])["analysis"]
    analysis_list.append(pd.DataFrame(res_dict))

analysis = pd.concat(analysis_list)
analysis.to_csv('./output/analysis.csv',index=False)

In [ ]:
import json
import pandas as pd
from importlib import reload

import lib.categorize_job_steps as this
reload(this)

job_steps = analysis["Job Step"]

res = this.chain.invoke({"main_job":selected_main_job, "job_performers": selected_job_performers, "job_steps":job_steps})
res_dict = (json.loads(res.additional_kwargs['function_call']['arguments']))
job_steps_categorized = pd.DataFrame(res_dict)

res_dict = (json.loads(res.additional_kwargs['function_call']['arguments']))
job_steps_categorized = pd.DataFrame(res_dict['analysis'])

analysis = pd.merge(analysis, job_steps_categorized[["Job Step", "Category"]], on='Job Step', how='outer')

In [ ]:
{
    "job performers":"System Administrator",
    "main_job": "Guarantee system reliability and uptime",
    "analysis": analysis.to_dict('records')
}